In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("C:\\Users\\paulo\\Documents\\Job Applications\\Data_Tests\\BFI - UChicago Dingel\\ny-demographics.csv")

In [3]:
data.head()

,geoid11,geoid11name,population,asian,black,hispanic,white
0,36001000100,"Census Tract 1, Albany County, New York",2139,55,1343,198,466
1,36001000200,"Census Tract 2, Albany County, New York",6046,54,4455,648,699
2,36001000300,"Census Tract 3, Albany County, New York",5888,211,2077,615,2740
3,36001000401,"Census Tract 4.01, Albany County, New York",2227,56,198,53,1896
4,36001000403,"Census Tract 4.03, Albany County, New York",4415,443,407,201,3275


In [4]:
# Double-checking that there are no missing values.
data.isna().sum()

geoid11        0
geoid11name    0
population     0
asian          0
black          0
hispanic       0
white          0
dtype: int64

In [6]:
# Create 5-digit county identifier
data['geoid5'] = (data['geoid11']//(1e06)).astype('int64')

In [8]:
# Create Census Tract, County Name, and State Name variables from geoid11name field
data[['Census Tract', 'County Name','State Name']] = data['geoid11name'].str.split(',',expand=True)

In [9]:
# Create name of county and state variable
data['geoid5name'] = data['County Name'] + ',' + data['State Name']
data['geoid5name']

0        Albany County, New York
1        Albany County, New York
2        Albany County, New York
3        Albany County, New York
4        Albany County, New York
                  ...           
4914      Yates County, New York
4915      Yates County, New York
4916      Yates County, New York
4917      Yates County, New York
4918      Yates County, New York
Name: geoid5name, Length: 4919, dtype: object

In [10]:
# I remove the "(rXXXXX)" values from the population column while keeping the true population counts in those rows.
data.loc[data['population'].str.contains('r'), 'population'] = data[data['population'].str.contains('r')]['population'].str.split('(',expand=True)[0]

In [11]:
# I change the population column's type to be an integer
data['population'] = data['population'].astype('int64')

In [12]:
data.head()

,geoid11,geoid11name,population,asian,black,hispanic,white,geoid5,Census Tract,County Name,State Name,geoid5name
0,36001000100,"Census Tract 1, Albany County, New York",2139,55,1343,198,466,36001,Census Tract 1,Albany County,New York,"Albany County, New York"
1,36001000200,"Census Tract 2, Albany County, New York",6046,54,4455,648,699,36001,Census Tract 2,Albany County,New York,"Albany County, New York"
2,36001000300,"Census Tract 3, Albany County, New York",5888,211,2077,615,2740,36001,Census Tract 3,Albany County,New York,"Albany County, New York"
3,36001000401,"Census Tract 4.01, Albany County, New York",2227,56,198,53,1896,36001,Census Tract 4.01,Albany County,New York,"Albany County, New York"
4,36001000403,"Census Tract 4.03, Albany County, New York",4415,443,407,201,3275,36001,Census Tract 4.03,Albany County,New York,"Albany County, New York"


In [13]:
# I only keep the rows in the dataset that have a positive population.
data = data[data['population'] > 0]

In [14]:
len(data[data['population'] > data['asian'] + data['black'] + data['hispanic'] + data['white']])

4840

In [15]:
len(data[data['population'] == data['asian'] + data['black'] + data['hispanic'] + data['white']])

30

In [16]:
len(data[data['population'] < data['asian'] + data['black'] + data['hispanic'] + data['white']])

0

First we note that the vast majority of the time the population count is higher than the sums of each race's total. I am going to assume that is due to the presence of individuals who chose to not identify their race. I further assume that those who do not identify their race have the same racial distribution as those that do identify their race (this is probably not a great assumption). Thus, I need to make a new population column of just those individuals who identified their race, and this is the column that I will use as the denominator when I determine the racial population shares in each county and census tract.

In [17]:
data['racial_pop'] = data['asian'] + data['black'] + data['hispanic'] + data['white']

<ipython-input-17-ac28e9acff72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['racial_pop'] = data['asian'] + data['black'] + data['hispanic'] + data['white']


In [18]:
# I create a dummy indicator for if the census track is the majority for a certain ethnicity (or no ethnicity)
data.loc[data['asian'] / data['racial_pop'] > 0.5,'asian_maj'] = 1
data.loc[data['asian'] / data['racial_pop'] <= 0.5,'asian_maj'] = 0
data.loc[data['black'] / data['racial_pop'] > 0.5,'black_maj'] = 1
data.loc[data['black'] / data['racial_pop'] <= 0.5,'black_maj'] = 0
data.loc[data['hispanic'] / data['racial_pop'] > 0.5,'hispanic_maj'] = 1
data.loc[data['hispanic'] / data['racial_pop'] <= 0.5,'hispanic_maj'] = 0
data.loc[data['white'] / data['racial_pop'] > 0.5,'white_maj'] = 1
data.loc[data['white'] / data['racial_pop'] <= 0.5,'white_maj'] = 0

C:\Users\paulo\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\paulo\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [20]:
data.loc[data['asian_maj'] + data['black_maj'] + data['hispanic_maj'] + data['white_maj']==1,'non_maj'] = 0
data.loc[data['asian_maj'] + data['black_maj'] + data['hispanic_maj'] + data['white_maj']==0,'non_maj'] = 1

C:\Users\paulo\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\paulo\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [21]:
# Add flag that census tract is populated
data['populated_flag'] = 1

<ipython-input-21-cc145887820f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['populated_flag'] = 1


In [22]:
cleaned_data = data.groupby(['geoid5','geoid5name']).sum()[['population','asian','black','hispanic','white','racial_pop','asian_maj','black_maj','hispanic_maj','white_maj','non_maj','populated_flag']].reset_index()
cleaned_data

,geoid5,geoid5name,population,asian,black,hispanic,white,racial_pop,asian_maj,black_maj,hispanic_maj,white_maj,non_maj,populated_flag
0,36001,"Albany County, New York",304204,14500,36396,14917,231152,296965,0.0,7.0,0.0,64.0,4.0,75
1,36003,"Allegany County, New York",48946,451,494,670,46701,48316,0.0,0.0,0.0,12.0,0.0,13
2,36005,"Bronx County, New York",1385108,47335,416695,741413,151209,1356652,0.0,54.0,218.0,37.0,27.0,336
3,36007,"Broome County, New York",200600,7019,8850,6778,173074,195721,0.0,0.0,0.0,55.0,0.0,55
4,36009,"Cattaraugus County, New York",80317,524,966,1345,73849,76684,0.0,0.0,0.0,20.0,0.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,36115,"Washington County, New York",63216,260,1734,1446,58996,62436,0.0,0.0,0.0,17.0,0.0,17
58,36117,"Wayne County, New York",93772,465,2743,3476,85318,92002,0.0,0.0,0.0,22.0,0.0,22
59,36119,"Westchester County, New York",949113,51123,126585,207032,544563,929303,0.0,17.0,22.0,142.0,40.0,221
60,36121,"Wyoming County, New York",42155,156,2233,1244,38042,41675,0.0,0.0,0.0,11.0,0.0,11


In [23]:
# This is just a quick check to make sure there are no errors where each geoid5 did not have a unique geoid5name that it was matched to.
len(data['geoid5'].unique())
# No such errors seemed to have occurred. There are as many unique rows in our cleaned dataset as there were unique geoid values in the original dataset.

62

In [24]:
# I create county-wide racial shares of the population.
cleaned_data['asian_share'] = cleaned_data['asian'] / cleaned_data['racial_pop']
cleaned_data['black_share'] = cleaned_data['black'] / cleaned_data['racial_pop']
cleaned_data['hispanic_share'] = cleaned_data['hispanic'] / cleaned_data['racial_pop']
cleaned_data['white_share'] = cleaned_data['white'] / cleaned_data['racial_pop']
cleaned_data.head()

,geoid5,geoid5name,population,asian,black,hispanic,white,racial_pop,asian_maj,black_maj,hispanic_maj,white_maj,non_maj,populated_flag,asian_share,black_share,hispanic_share,white_share
0,36001,"Albany County, New York",304204,14500,36396,14917,231152,296965,0.0,7.0,0.0,64.0,4.0,75,0.048827,0.122560,0.050232,0.778381
1,36003,"Allegany County, New York",48946,451,494,670,46701,48316,0.0,0.0,0.0,12.0,0.0,13,0.009334,0.010224,0.013867,0.966574
2,36005,"Bronx County, New York",1385108,47335,416695,741413,151209,1356652,0.0,54.0,218.0,37.0,27.0,336,0.034891,0.307150,0.546502,0.111457
3,36007,"Broome County, New York",200600,7019,8850,6778,173074,195721,0.0,0.0,0.0,55.0,0.0,55,0.035862,0.045217,0.034631,0.884289
4,36009,"Cattaraugus County, New York",80317,524,966,1345,73849,76684,0.0,0.0,0.0,20.0,0.0,20,0.006833,0.012597,0.017540,0.963030


In [25]:
# I rename certain columns to have the desired names.
cleaned_data['tracts'] = cleaned_data['populated_flag']
cleaned_data['asian_majority_tracts'] = cleaned_data['asian_maj']
cleaned_data['black_majority_tracts'] = cleaned_data['black_maj']
cleaned_data['hispanic_majority_tracts'] = cleaned_data['hispanic_maj']
cleaned_data['white_majority_tracts'] = cleaned_data['white_maj']
cleaned_data['nonmajority_majority_tracts'] = cleaned_data['non_maj']

In [26]:
# I remove unneeded columns
del cleaned_data['asian']
del cleaned_data['black']
del cleaned_data['hispanic']
del cleaned_data['white']
del cleaned_data['racial_pop']
del cleaned_data['asian_maj']
del cleaned_data['black_maj']
del cleaned_data['hispanic_maj']
del cleaned_data['white_maj']
del cleaned_data['non_maj']
del cleaned_data['populated_flag']

In [27]:
cleaned_data.sort_values('geoid5')

,geoid5,geoid5name,population,asian_share,black_share,hispanic_share,white_share,tracts,asian_majority_tracts,black_majority_tracts,hispanic_majority_tracts,white_majority_tracts,nonmajority_majority_tracts
0,36001,"Albany County, New York",304204,0.048827,0.122560,0.050232,0.778381,75,0.0,7.0,0.0,64.0,4.0
1,36003,"Allegany County, New York",48946,0.009334,0.010224,0.013867,0.966574,13,0.0,0.0,0.0,12.0,0.0
2,36005,"Bronx County, New York",1385108,0.034891,0.307150,0.546502,0.111457,336,0.0,54.0,218.0,37.0,27.0
3,36007,"Broome County, New York",200600,0.035862,0.045217,0.034631,0.884289,55,0.0,0.0,0.0,55.0,0.0
4,36009,"Cattaraugus County, New York",80317,0.006833,0.012597,0.017540,0.963030,20,0.0,0.0,0.0,20.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,36115,"Washington County, New York",63216,0.004164,0.027772,0.023160,0.944904,17,0.0,0.0,0.0,17.0,0.0
58,36117,"Wayne County, New York",93772,0.005054,0.029815,0.037782,0.927349,22,0.0,0.0,0.0,22.0,0.0
59,36119,"Westchester County, New York",949113,0.055012,0.136215,0.222782,0.585991,221,0.0,17.0,22.0,142.0,40.0
60,36121,"Wyoming County, New York",42155,0.003743,0.053581,0.029850,0.912825,11,0.0,0.0,0.0,11.0,0.0


In [28]:
cleaned_data.to_csv("C:\\Users\\paulo\\Documents\\Job Applications\\Data_Tests\\BFI - UChicago Dingel\\opheim_dataset.csv")